<a href="https://colab.research.google.com/github/harnalashok/deeplearning/blob/main/reusing_trained_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Last amended: 26th January, 2021
# My folder: https://github.com/harnalashok/deeplearning
# Objective: 
#             i) Reuse layers from one neural network in another.
#             ii)Ist model was trained using a different set of 
#                class labels than present in the IInd case
#
# Ref:
#      Aurelien Book: Page 347 
#      Save and Laod models: https://www.tensorflow.org/guide/keras/save_and_serialize

**Problem**<br>
We will take fashion-mnist data. This data contains ten items. We will filter out data for two of the items--shirts and sandals. Prepare a predictive model with remaining eight items.<br>
After predictive model is made, we will use the trained layers of this model in another NN model to make predictions for the remaining two items. That is, we will reuse trained layers from the first model in the IInd model.

In [13]:
# 1.0 Call libraries
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import plot_model
import numpy as np

In [14]:
# 1.1 Display multiple commands outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [15]:
# 2.0 Get split data
(X_train, y_train), (X_test,y_test) = fashion_mnist.load_data()

In [ ]:
# 2.1 Shape of train data
X_train.shape    # (60000, 28, 28)
y_train.shape    # (60000,)

In [17]:
# 2.2 Fashion items
items = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [18]:
# 2.3 Normalise train/test data
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
# 2.4 We filter our data further
#     Put Shirts and Sandals in one dataset
#     And remaining 8 items in another dataset.
#     Shirts and Sandals are labelled as '5' & '6'

## Get train/test data with shirts and sandals:
# 2.4.1
ss_train = X_train[np.logical_or (y_train == 5,y_train == 6), :, :]
ss_train.shape   # (12000, 28, 28)
# 2.4.2
ss_y_train = y_train[np.logical_or (y_train == 5,y_train == 6)]
ss_y_train.shape  # (12000,)

# 2.4.3
ss_test = X_test[np.logical_or (y_test == 5,y_test == 6), :, :]
ss_test.shape    # (2000, 28, 28)
# 2.4.4
ss_y_test = y_test[np.logical_or (y_test == 5,y_test == 6)]
ss_y_test.shape   # (2000,)

In [ ]:
# 3.0 Change labels of shirts and sandals
#     in ss_y_train/ss_y_test to 1 and 0
#     from present 6 and 5

ss_y_train[ss_y_train == 6] = 1
ss_y_train[ss_y_train == 5] = 0

# 3.1 Recheck 
ss_y_train

In [ ]:
# 4.0 Filter train/test data for remaining 8 items
# 4.1 train data
data_train = X_train[ np.logical_and (y_train != 5 , y_train !=6), :,:]
data_train.shape  # (48000, 28, 28)

# 4.2 'train' labels
data_y_train = y_train[ np.logical_and (y_train != 5 , y_train !=6)]
data_y_train.shape  # (48000,)

# 4.3 Filter 'test' data
data_test = X_test[ np.logical_and (y_test != 5 , y_test !=6), :,:]
data_test.shape   # (8000, 28, 28)
# 4.4 Filter data with 'test' labels
data_y_test = y_test[ np.logical_and (y_test != 5 , y_test !=6)]
data_y_test.shape  # (8000,)

In [22]:
# 4.5 Reshape train/test data

#     8-items data
data_train = data_train.reshape(-1,784)
data_test = data_test.reshape(-1,784)

# 4.6 2-items data
ss_train = ss_train.reshape(-1,784)
ss_test = ss_test.reshape(-1,784)

In [ ]:
# 5.0 Our Ist model. No frills

units =100

# 5.1 Input layer
input_one = keras.layers.Input(shape = (784,))

# 5.2 Hidden layers
x = keras.layers.Dense(units = units, activation = "selu", name = "Layer_a")(input_one)
x = keras.layers.Dense(units = units, activation = "selu", name = "Layer_b")(x)
x = keras.layers.Dense(units = units, activation = "selu", name = "Layer_c")(x)

# 5.3 Output layer
out_one = keras.layers.Dense(units = 10, activation = 'softmax', name = "output_layer")(x)

# 5.4 Design model
model_one = Model(inputs = [input_one], outputs = [out_one])

# 5.5 Compile model
model_one.compile(loss = "sparse_categorical_crossentropy", metrics = "accuracy")

# 5.6 Fit the model
model_one.fit(data_train,data_y_train, epochs = 20)

In [ ]:
# 5.7 Model summary
model_one.summary()

In [ ]:
# 5.8 Plot the model
plot_model(model_one)

In [ ]:
# 6.1 Check folders under /home
! ls /home


In [ ]:
# 6.0 Save this model in a folder. 
#     Folder() will be created if 
#     they do not exist:
#  Ref: https://www.tensorflow.org/api_docs/python/tf/keras/models/save_model

path = "/home/ashok/mymodel"

tf.keras.models.save_model(
                           model_one,
                           path,
                           include_optimizer=True,  # Default
                           overwrite = True         # Default
                           )


In [28]:
# 6.1 Let us have a look at what was saved:

! ls /home/ashok/mymodel


assets	saved_model.pb	variables


In [29]:
# 6.2 Now that model is saved, 
#     we can delete it from RAM
del(model_one)

In [ ]:
# 6.3 Check if 'model' is deleted
model_one.layers

In [31]:
# 6.4 Let us load back the saved model 
model = keras.models.load_model(path)

In [ ]:
# 6.5 Check model summary
model.summary()

There are two ways of creating our new model that will reuse the trained layers of our old model.

In [33]:
## 7.0
# Method 1: Build a fresh model having
#           first three layers with 
#           the same configuration 
#           as in the earlier model
#           Our intention is to REUSE 
#           learnt weights in the Ist three 
#           layers in the new model


units =100
input_l = keras.layers.Input(shape = (784,))
y = keras.layers.Dense(units = units, activation = "selu", name = "a")(input_l)
y = keras.layers.Dense(units = units, activation = "selu", name = "b")(y)
y = keras.layers.Dense(units = units, activation = "selu", name = "c")(y)


In [34]:
# 7.1 Now add to it our own layers

y = keras.layers.Dense(units = 20, activation = 'selu', name = "x1")(y)
y = keras.layers.Dense(units = 8, activation = 'relu', name = "x2")(y)
out_l = keras.layers.Dense(units = 1, activation = 'sigmoid', name = "out_layer")(y)

In [ ]:
# 7.2 Create model
model_l = Model(inputs = [input_l], outputs = [out_l])
model_l.compile(loss = "binary_crossentropy", metrics = ["accuracy"])
model_l.summary()

In [36]:
# 8.0 Let us transfer weights in this model
#     from weights from our earlier learned-model
#     After transferring weights, let us also freeze
#     the layer-weights

for i in range(4):
  model_l.layers[i].set_weights(model.layers[i].get_weights())
  model_l.layers[i].trainable = False

In [37]:
# 8.1 As our last layer is Dense(1) with sigmoid activation
#     therefore, target data has to have shape of (None,1), rather
#     than (None,) as at present

ss_y_train = ss_y_train.reshape(-1,1)
ss_y_train.shape

(12000, 1)

In [ ]:
# 8.2 Before we proceed for training
#     let us see which layers are trainable,
#     and which not

for layer in model_l.layers:
  print(layer.name,":",layer.trainable)

In [ ]:
# 8.3 Begin training
model_l.fit(ss_train ,ss_y_train, epochs = 20)

In [40]:
## 9.0
# Method 2:
#          Using the last but one layer
#          create a Sequential model

model_IInd = keras.models.Sequential(model.layers[:-1])

# 9.1 And add to it our own layers
model_IInd.add(keras.layers.Dense(units = 20, activation = 'selu', name = "x1"))
model_IInd.add(keras.layers.Dense(units = 8, activation = 'relu', name = "x2"))
model_IInd.add(keras.layers.Dense(units = 1, activation = 'sigmoid', name = "out_layer"))


In [41]:
# 9.2 Compile the model
model_IInd.compile(loss = "binary_crossentropy", metrics = ['accuracy'])


In [42]:
# 9.3 Let us freeze weights in some layers 

for i in range(4):
  model_IInd.layers[i].trainable = False

In [ ]:
# 9.4 Before we proceed for training
#     let us see which layers are trainable,
#     and which not

for layer in model_IInd.layers:
  print(layer.name,":",layer.trainable)

In [ ]:
# 9.5 Begin training now:
model_IInd.fit(ss_train,ss_y_train, epochs = 20)

In [ ]:
######## I am done ##########